In [1]:
import numpy as np 
import open3d as o3d
import time
import logging
import math

sample_directory = r"C:\Repo\SCAN2BIM-python\Samples\TrainingdataCreation"

timestr = time.strftime("%Y%m%d-%H%M%S")
voxel_size_ref = 0.05 # voxel size for the downsampling
voxel_size_target = 0.05
init_radius = 0.15 # initial neirest neighbour search radius
c2c_treshold = 0.15 # All points with a c2c-distance larger than this treshold will be labeled as clutter
search_treshold = c2c_treshold # Max distance for normal comparison 

LOA_filename = "LOA_" + str(voxel_size_target) + "_" + timestr + ".txt"
timing_filename = "Timing_" + str(voxel_size_target) + "_" + timestr + ".txt"

LOA_location = sample_directory + "\\" + LOA_filename
timing_location = sample_directory + "\\" + timing_filename



In [2]:
Zero_time = time.perf_counter()

print("Import meshes ...")

print("... CEILING ...")
mesh_ceiling = o3d.io.read_triangle_mesh(r"C:\Data\Temp\TC-Ghent\BIM\OBJ\ceiling.obj")
print(mesh_ceiling)
ceiling_points = round(mesh_ceiling.get_surface_area() * 1000)
print("Convert Mesh to pointcloud")
meshpcd_ceiling = mesh_ceiling.sample_points_uniformly(number_of_points = ceiling_points, use_triangle_normal=True)
print(meshpcd_ceiling)
# o3d.visualization.draw_geometries([meshpcd_ceiling])
print("Voxel downsampling mesh with voxel size: %s m" % voxel_size_ref )
meshpcd_ceiling = meshpcd_ceiling.voxel_down_sample(voxel_size_ref)
print(meshpcd_ceiling)
# o3d.visualization.draw_geometries([meshpcd_ceiling])

print("... FLOOR ...")
mesh_floor = o3d.io.read_triangle_mesh(r"C:\Data\Temp\TC-Ghent\BIM\OBJ\floor.obj")
print(mesh_floor)
print("Convert Mesh to pointcloud")
floor_points = round(mesh_floor.get_surface_area() * 1000)
meshpcd_floor = mesh_floor.sample_points_uniformly(number_of_points = floor_points, use_triangle_normal=True)
print(meshpcd_floor)
# o3d.visualization.draw_geometries([meshpcd_floor])
print("Voxel downsampling mesh with voxel size: %s m" % voxel_size_ref )
meshpcd_floor = meshpcd_floor.voxel_down_sample(voxel_size_ref)
print(meshpcd_floor)
# o3d.visualization.draw_geometries([meshpcd_floor])

print("... WALL ...")
mesh_wall = o3d.io.read_triangle_mesh(r"C:\Data\Temp\TC-Ghent\BIM\OBJ\wall.obj")
print(mesh_wall)
print("Convert Mesh to pointcloud")
wall_points = round(mesh_wall.get_surface_area() * 1000)
meshpcd_wall = mesh_wall.sample_points_uniformly(number_of_points = wall_points, use_triangle_normal=True)
print(meshpcd_wall)
# o3d.visualization.draw_geometries([meshpcd_wall])
print("voxel downsampling mesh with voxel size: %s m" % voxel_size_ref )
meshpcd_wall = meshpcd_wall.voxel_down_sample(voxel_size_ref)
print(meshpcd_wall)
# o3d.visualization.draw_geometries([meshpcd_wall])

print("... BEAM ...")
mesh_beam = o3d.io.read_triangle_mesh(r"C:\Data\Temp\TC-Ghent\BIM\OBJ\beam.obj")
print(mesh_beam)
print("Convert Mesh to pointcloud")
beam_points = round(mesh_beam.get_surface_area() * 1000)
meshpcd_beam = mesh_beam.sample_points_uniformly(number_of_points = beam_points, use_triangle_normal=True)
print(meshpcd_beam)
# o3d.visualization.draw_geometries([meshpcd_beam])
print("voxel downsampling mesh with voxel size: %s m" % voxel_size_ref )
meshpcd_beam = meshpcd_beam.voxel_down_sample(voxel_size_ref)
print(meshpcd_beam)
# o3d.visualization.draw_geometries([meshpcd_beam])

print("... COLUMN ...")
mesh_column = o3d.io.read_triangle_mesh(r"C:\Data\Temp\TC-Ghent\BIM\OBJ\column.obj")
print(mesh_column)
print("Convert Mesh to pointcloud")
column_points = round(mesh_column.get_surface_area() * 1000)
meshpcd_column = mesh_column.sample_points_uniformly(number_of_points = column_points, use_triangle_normal=True)
print(meshpcd_column)
# o3d.visualization.draw_geometries([meshpcd_column])
print("voxel downsampling mesh with voxel size: %s m" % voxel_size_ref )
meshpcd_column = meshpcd_column.voxel_down_sample(voxel_size_ref)
print(meshpcd_column) 
# o3d.visualization.draw_geometries([meshpcd_column])

t1 = time.perf_counter()
dt1 = t1 - Zero_time
print("Processed meshes in %s s" % dt1)
logging.info("Imported and sampled reference meshes in %s s.\n" % dt1)


Import meshes ...
... CEILING ...
[Open3D WARNING] Unable to load file C:\Data\Temp\TC-Ghent\BIM\OBJ\ceiling.obj with ASSIMP
TriangleMesh with 0 points and 0 triangles.
Convert Mesh to pointcloud


RuntimeError: [Open3D ERROR] [SamplePointsUniformly] number_of_points <= 0

In [3]:
t1 = time.perf_counter()

# Merge pointclouds from the meshes to 1 pointcloud and save labeles to a separate array
# FAST method
print("Merge refernce clouds to one pointcloud ...")

labels =[]
label_id = [ "Ceiling", "Floor", "Wall", "Beam", "Column", "Clutter"]

length_ceiling = len(meshpcd_ceiling.points)
i = 0
while i < length_ceiling:
    labels.append("0")
    i =  i+1

length_floor = len(meshpcd_floor.points)
i = 0
while i < length_floor:
    labels.append("1")
    i =  i+1

length_wall = len(meshpcd_wall.points)
i = 0
while i < length_wall:
    labels.append("2")
    i =  i+1

length_beam = len(meshpcd_beam.points)
i = 0
while i < length_beam:
    labels.append("3")
    i =  i+1

length_column = len(meshpcd_column.points)
i = 0
while i < length_column:
    labels.append("4")
    i =  i+1

# mesh_ceiling_t = o3d.t.geometry.PointCloud.from_legacy_pointcloud(meshpcd_ceiling)

meshpcd = o3d.geometry.PointCloud()
meshpcd.__iadd__(meshpcd_ceiling)
print(meshpcd)
meshpcd.__iadd__(meshpcd_floor)
print(meshpcd)
meshpcd.__iadd__(meshpcd_wall)
print(meshpcd)
meshpcd.__iadd__(meshpcd_beam)
print(meshpcd)
meshpcd.__iadd__(meshpcd_column)
print(meshpcd)
logging.info(str(meshpcd))

print("Reference cloud contains normals: %s" % meshpcd.has_normals())
print("%s / %s" %(len(labels), len(meshpcd.points)))
t2 = time.perf_counter()
dt2 = t2 - t1
print("Created reference cloud from meshes in %s s" % dt2)

Merge refernce clouds to one pointcloud ...
PointCloud with 276926 points.
PointCloud with 579267 points.
PointCloud with 1039048 points.
PointCloud with 1078560 points.
PointCloud with 1085675 points.
Reference cloud contains normals: True
1085675 / 1085675
Created reference cloud from meshes in 0.19174480000000038 s


In [4]:
# print(np.asarray(meshpcd.points[1:10]))

In [5]:
# o3d.visualization.draw_geometries([meshpcd])

In [6]:
t2 = time.perf_counter()

print("Load targeted pointcloudpoint cloud ...")

pointcloud_location = r"C:\Data\Temp\TC-Ghent\Pointclouds\PCD\Labo-Beton-1.pcd"
pcd = o3d.io.read_point_cloud(pointcloud_location)


t3 = time.perf_counter()
dt3 = t3 - t2
print("Target cloud loaded in %s s" % dt3)
print(pcd)

print("Downsampling pointcloud with voxel size: %s m" % voxel_size_target)
pcd = pcd.voxel_down_sample(voxel_size_target)
# tar_voxel.paint_uniform_color([0.5,0.5,0.5]) #Grey

t4 = time.perf_counter()
dt4 = t4 - t3

print("Downsampled the target pointcloud in %s s" % dt4)
print(pcd)
print("Target pointcloud has normals: %s " % pcd.has_normals())


Load targeted pointcloudpoint cloud ...
Target cloud loaded in 8.392952100000002 s
PointCloud with 70176074 points.
Downsampling pointcloud with voxel size: 0.05 m
Downsampled the target pointcloud in 3.461288999999997 s
PointCloud with 715870 points.
Target pointcloud has normals: False 


In [7]:
pcd.estimate_normals()

# print("mesh normaal: %s " %meshpcd.normals[5425])
# print("mesh normaal norm: %s "% np.linalg.norm(meshpcd.normals[5]))
# print("point normaal: %s " %pcd.normals[5425])
# print("point normaal norm: %s "% np.linalg.norm(pcd.normals[5]))

In [8]:
# print("Target pointcloud has normals: %s " % pcd.has_normals())

In [9]:
t4 = time.perf_counter()

c2c = pcd.compute_point_cloud_distance(meshpcd)

t5 = time.perf_counter()
dt5 = t5 - t4

print("Cloud 2 cloud calculated in %s s" % dt5)
print("%s / %s" %(len(c2c), len(pcd.points)))


Cloud 2 cloud calculated in 1.0548893999999969 s
715870 / 715870


In [10]:
t5 = time.perf_counter()

# Filter pointcloud on c2c distances
# FAST method

i=0
percentage = 0
Inlier_1_indeces = []
Outlier_indeces = []

while i < len(c2c):
    if c2c[i] <= c2c_treshold:
        np.asarray(pcd.colors)[i] = [0,1,0]
        Inlier_1_indeces.append(i)
    elif c2c[i] > c2c_treshold:
        np.asarray(pcd.colors)[i] = [1,0,0]
        Outlier_indeces.append(i)

    progress = i/len(c2c)*100
    
    if progress >= percentage:
        timing = time.perf_counter()
        dtiming = timing - t5
        hours = math.floor(dtiming/3600)
        minutes = math.floor(dtiming/60 - hours*60)
        seconds = dtiming - hours*3600 - minutes*60
        print("progress: %s %% - %sh %smin %ss" %(round(progress),hours, minutes, seconds))
        percentage = percentage + 5
    i = i+1

# Inlier_pcd = pcd.select_by_index(Inlier_indeces)


t6 = time.perf_counter()
dt6 = t6 - t5
print("split in %s s" % dt6)

progress: 0 % - 0h 0min 0.00023600000000101318s
progress: 5 % - 0h 0min 0.2945871000000011s
progress: 10 % - 0h 0min 0.5613472000000002s
progress: 15 % - 0h 0min 0.8295050999999987s
progress: 20 % - 0h 0min 1.0961372000000011s
progress: 25 % - 0h 0min 1.381548200000001s
progress: 30 % - 0h 0min 1.649527599999999s
progress: 35 % - 0h 0min 1.9337526000000018s
progress: 40 % - 0h 0min 2.2155118000000016s
progress: 45 % - 0h 0min 2.4940390999999984s
progress: 50 % - 0h 0min 2.7689598999999987s
progress: 55 % - 0h 0min 3.0456670999999993s
progress: 60 % - 0h 0min 3.3248099999999994s
progress: 65 % - 0h 0min 3.6066763s
progress: 70 % - 0h 0min 3.892632599999999s
progress: 75 % - 0h 0min 4.1722633999999985s
progress: 80 % - 0h 0min 4.4564927999999995s
progress: 85 % - 0h 0min 4.7250136000000005s
progress: 90 % - 0h 0min 4.999245200000001s
progress: 95 % - 0h 0min 5.442054200000001s
split in 5.904917900000001 s


In [11]:
print(len(Inlier_1_indeces))

456638


In [12]:
# o3d.visualization.draw_geometries([tar_voxel])

In [13]:
# o3d.visualization.draw_geometries([Inlier_pcd])

De Dichtste mesh vinden 

Normaal van het punt vergelijken met de normaal van de mesh

Afwijkende normale ditchen 


In [14]:
# print(labels)

In [15]:
# print(" %s/%s" %(len(labels),len(meshpcd.points)))

In [34]:
t6 = time.perf_counter()

# Filter pointcloud on normals and assign labels.


i=0
percentage = 0
Inlier_2_indeces = []
Final_inlier_labels = []
Final_LOAs = []

print("creating a KD-tree")
meshpcd_tree = o3d.geometry.KDTreeFlann(meshpcd)#Create KD tree index

print("Finished creating a KD-tree")
progress = 0
count = 0
radius = init_radius

for i in Inlier_1_indeces:
    [k, idx, d] = meshpcd_tree.search_radius_vector_3d(pcd.points[i], init_radius) #Neighbour Search radius 10cm 
    # print("Selected points in the neighbourhood")
    # meshpcd_sample_points = np.asarray(meshpcd.points)[idx[:], :]
    # meshpcd_sample_normals = np.asarray(meshpcd.normals)[idx[:], :]
    Not_found = True
    i1=0
    while Not_found and i1 < len(idx) and len(idx) > 0:
        if np.abs(np.dot(np.asarray(pcd.normals[i]), np.asarray(meshpcd.normals)[idx[i1],:])) > 0.9 or np.abs(d[i1]) < c2c_treshold/5:
            Not_found = False
            Inlier_2_indeces.append(i)
            Final_inlier_labels.append(labels[idx[i1]])
            Final_LOAs.append(d[i1]) 
        i1 = i1+1

    if Not_found:
        Outlier_indeces.append(i)

    count = count + 1
    progress = count/len(Inlier_1_indeces)*100
    
    if progress >= percentage:
        timing = time.perf_counter()
        dtiming = timing - t6
        hours = math.floor(dtiming/3600)
        minutes = math.floor(dtiming/60 - hours*60)
        seconds = dtiming - hours*3600 - minutes*60
        print("progress: %s %% - %sh %smin %ss" %(round(progress),hours, minutes, seconds))
        percentage = percentage + 5


print("extracting relevant points")

Final_inlier_pcd = pcd.select_by_index(Inlier_2_indeces)
print(Final_inlier_pcd)
Clutter_pcd = pcd.select_by_index(Outlier_indeces)
print(Clutter_pcd)

t7 = time.perf_counter()
dt7 = t7 - t6
print("filtered data in %s s" % dt7)
logging.info("Fine filtering took %s s" % dt7)

creating a KD-tree
Finished creating a KD-tree
progress: 0 % - 0h 0min 0.3658499999999947s
progress: 5 % - 0h 0min 1.1460567000000026s
progress: 10 % - 0h 0min 1.9240213999999867s
progress: 15 % - 0h 0min 2.7112658000000067s
progress: 20 % - 0h 0min 3.4804342999999847s
progress: 25 % - 0h 0min 4.268990599999995s
progress: 30 % - 0h 0min 5.0593470999999965s
progress: 35 % - 0h 0min 5.831893699999995s
progress: 40 % - 0h 0min 6.608428400000008s
progress: 45 % - 0h 0min 7.400421899999998s
progress: 50 % - 0h 0min 8.178049999999985s
progress: 55 % - 0h 0min 8.943799100000007s
progress: 60 % - 0h 0min 9.709979299999986s
progress: 65 % - 0h 0min 10.49901779999999s
progress: 70 % - 0h 0min 11.256884499999984s
progress: 75 % - 0h 0min 12.016461800000002s
progress: 80 % - 0h 0min 12.77158209999999s
progress: 85 % - 0h 0min 13.510319199999998s
progress: 90 % - 0h 0min 14.244270400000005s
progress: 95 % - 0h 0min 14.982376999999985s
progress: 100 % - 0h 0min 15.73341959999999s
extracting relevant

In [35]:
# print(" %s/%s" %(len(Final_inlier_labels),len(Final_inlier_pcd.points)))

In [36]:
# print(Final_inlier_labels)
t7 = time.perf_counter()

In [37]:
t7 = time.perf_counter()

i=0
percentage = 0

Class_0_indeces = []
Class_0_points = 0
Class_0_LOA30 = 0
Class_0_LOA20 = 0
Class_0_LOA10 = 0

Class_1_indeces = []
Class_1_points = 0
Class_1_LOA30 = 0
Class_1_LOA20 = 0
Class_1_LOA10 = 0

Class_2_indeces = []
Class_2_points = 0
Class_2_LOA30 = 0
Class_2_LOA20 = 0
Class_2_LOA10 = 0

Class_3_indeces = []
Class_3_points = 0
Class_3_LOA30 = 0
Class_3_LOA20 = 0
Class_3_LOA10 = 0

Class_4_indeces = []
Class_4_points = 0
Class_4_LOA30 = 0
Class_4_LOA20 = 0
Class_4_LOA10 = 0

Class_clutter_points = len(Clutter_pcd.points)
Clutter_pcd.paint_uniform_color([0.5,0.5,0.5]) #Grey
Final_inlier_pcd.paint_uniform_color([0.5,0.5,0.5]) #Grey

while i < len(np.asarray(Final_inlier_pcd.points)):
    if Final_inlier_labels[i] == "0":
        np.asarray(Final_inlier_pcd.colors)[i] = [0,0,0]
        Class_0_indeces.append(i)
        Class_0_points = Class_0_points + 1
        dist = Final_LOAs[i]
        if dist <= 0.015:
            Class_0_LOA30 = Class_0_LOA30 + 1
        elif dist > 0.015 and dist <=0.05:
            Class_0_LOA20 = Class_0_LOA20 + 1
        elif dist > 0.05:
            Class_0_LOA10 = Class_0_LOA10 + 1

    elif Final_inlier_labels[i] == "1":
        np.asarray(Final_inlier_pcd.colors)[i] = [1,0,0]
        Class_1_indeces.append(i)
        Class_1_points = Class_1_points + 1
        dist = Final_LOAs[i]
        if dist <= 0.015:
            Class_1_LOA30 = Class_1_LOA30 + 1
        elif dist > 0.015 and dist <=0.05:
            Class_1_LOA20 = Class_1_LOA20 + 1
        elif dist > 0.05:
            Class_1_LOA10 = Class_1_LOA10 + 1

    elif Final_inlier_labels[i] == "2":
        np.asarray(Final_inlier_pcd.colors)[i] = [0,1,0]
        Class_2_indeces.append(i)
        Class_2_points = Class_2_points + 1
        dist = Final_LOAs[i]
        if dist <= 0.015:
            Class_2_LOA30 = Class_2_LOA30 + 1
        elif dist > 0.015 and dist <=0.05:
            Class_2_LOA20 = Class_2_LOA20 + 1
        elif dist > 0.05:
            Class_2_LOA10 = Class_2_LOA10 + 1

    elif Final_inlier_labels[i] == "3":
        np.asarray(Final_inlier_pcd.colors)[i] = [0,0,1]
        Class_3_indeces.append(i)
        Class_3_points = Class_3_points + 1
        dist = Final_LOAs[i]
        if dist <= 0.015:
            Class_3_LOA30 = Class_3_LOA30 + 1
        elif dist > 0.015 and dist <=0.05:
            Class_3_LOA20 = Class_3_LOA20 + 1
        elif dist > 0.05:
            Class_3_LOA10 = Class_3_LOA10 + 1

    elif Final_inlier_labels[i] == "4":
        np.asarray(Final_inlier_pcd.colors)[i] = [1,1,0]
        Class_4_indeces.append(i)
        Class_4_points = Class_4_points + 1
        dist = Final_LOAs[i]
        if dist <= 0.015:
            Class_4_LOA30 = Class_4_LOA30 + 1
        elif dist > 0.015 and dist <=0.05:
            Class_4_LOA20 = Class_4_LOA20 + 1
        elif dist > 0.05:
            Class_4_LOA10 = Class_4_LOA10 + 1

    progress = i/len(np.asarray(Final_inlier_pcd.points))*100
    if progress >= percentage:
        timing = time.perf_counter()
        dtiming = timing - t7
        hours = math.floor(dtiming/3600)
        minutes = math.floor(dtiming/60 - hours*60)
        seconds = dtiming - hours*3600 - minutes*60
        print("progress: %s %% - %sh %smin %ss" %(round(progress),hours, minutes, seconds))
        percentage = percentage + 5
    i= i+1

print("...DONE...")
t8 = time.perf_counter()
dt8 = t8 - t7
print("calculated LOA specs in %s s" % dt8)


progress: 0 % - 0h 0min 0.009023600000006127s
progress: 5 % - 0h 0min 0.2770992000000092s
progress: 10 % - 0h 0min 0.5504757000000211s
progress: 15 % - 0h 0min 0.8274843000000089s
progress: 20 % - 0h 0min 1.098959700000023s
progress: 25 % - 0h 0min 1.3987288000000149s
progress: 30 % - 0h 0min 1.6728836000000058s
progress: 35 % - 0h 0min 1.95646720000002s
progress: 40 % - 0h 0min 2.2253709999999955s
progress: 45 % - 0h 0min 2.5044524000000195s
progress: 50 % - 0h 0min 2.7836077999999986s
progress: 55 % - 0h 0min 3.049574000000007s
progress: 60 % - 0h 0min 3.3175547000000165s
progress: 65 % - 0h 0min 3.5860790000000122s
progress: 70 % - 0h 0min 3.849684400000001s
progress: 75 % - 0h 0min 4.119656399999997s
progress: 80 % - 0h 0min 4.3856633000000045s
progress: 85 % - 0h 0min 4.649302300000016s
progress: 90 % - 0h 0min 4.932609100000008s
progress: 95 % - 0h 0min 5.201901700000008s
...DONE...
calculated LOA specs in 5.4685995000000105 s


In [38]:
# t8 = time.perf_counter()

# result_directory = sample_directory + "\\Results_" + timestr
# os.mkdir(result_directory)

# print("Class 0:  %s " % label_id[0])
# Class_0_pcd = Final_inlier_pcd.select_by_index(Class_0_indeces)
# print(Class_0_pcd)
# Class_0_filename = "Cloud_" + str(voxel_size_target) + "_" + label_id[0] + ".pcd"
# Class_0_location = result_directory +"\\" + Class_0_filename
# o3d.io.write_point_cloud(Class_0_location, Class_0_pcd, print_progress=True)
# print("Saved to:  %s " % Class_0_location)

# print("Class 1:  %s " % label_id[1])
# Class_1_pcd = Final_inlier_pcd.select_by_index(Class_1_indeces)
# print(Class_1_pcd)
# Class_1_filename = "Cloud_" + str(voxel_size_target) + "_" + label_id[1] + ".pcd"
# Class_1_location = result_directory +"\\" + Class_1_filename
# o3d.io.write_point_cloud(Class_1_location, Class_1_pcd, print_progress=True)
# print("Saved to:  %s " % Class_1_location)

# print("Class 2:  %s " % label_id[2])
# Class_2_pcd = Final_inlier_pcd.select_by_index(Class_2_indeces)
# print(Class_2_pcd)
# Class_2_filename = "Cloud_" + str(voxel_size_target) + "_" + label_id[2] + ".pcd"
# Class_2_location = result_directory +"\\" + Class_2_filename
# o3d.io.write_point_cloud(Class_2_location, Class_2_pcd, print_progress=True)
# print("Saved to:  %s " % Class_2_location)

# print("Class 3:  %s " % label_id[3])
# Class_3_pcd = Final_inlier_pcd.select_by_index(Class_3_indeces)
# print(Class_3_pcd)
# Class_3_filename = "Cloud_" + str(voxel_size_target) + "_" + label_id[3] + ".pcd"
# Class_3_location = result_directory +"\\" + Class_3_filename
# o3d.io.write_point_cloud(Class_3_location, Class_3_pcd, print_progress=True)
# print("Saved to:  %s " % Class_3_location)

# print("Class 4:  %s " % label_id[4])
# Class_4_pcd = Final_inlier_pcd.select_by_index(Class_4_indeces)
# print(Class_4_pcd)
# Class_4_filename = "Cloud_" + str(voxel_size_target) + "_" + label_id[4] + ".pcd"
# Class_4_location = result_directory +"\\" + Class_4_filename
# o3d.io.write_point_cloud(Class_1_location, Class_4_pcd, print_progress=True)
# print("Saved to:  %s " % Class_4_location)

# print("Class Clutter")
# print(Clutter_pcd)
# Class_Clutter_filename = "Cloud_" + str(voxel_size_target) + "_Clutter"  + ".pcd"
# Class_Clutter_location = result_directory +"\\" + Class_Clutter_filename
# o3d.io.write_point_cloud(Class_Clutter_location, Clutter_pcd, print_progress=True)
# print("Saved to:  %s " % Class_Clutter_location)

# print("General Cloud")
# pcd_filename = "GeneralCloud_" + str(voxel_size_target) + "_" + timestr + ".pcd"
# pcd_location = result_directory +"\\" + pcd_filename
# o3d.io.write_point_cloud(pcd_location, Final_inlier_pcd, print_progress=True)
# print("Saved to:  %s " % pcd_location)

# t9 = time.perf_counter()
# dt9 = t9 - t8
# print("Saved clouds in %s s" % dt9)

In [39]:
# LOA_file = open(LOA_location, "w")

In [40]:
# t9 = time.perf_counter()


# LOA_file = open(LOA_location, "w")

# LOA_file.write("LOA specs\n")
# LOA_file.write("I. Pointcloud specifications:\n\n")
# LOA_file.write("    Voxel size reference: %sm\n" % voxel_size_ref)
# LOA_file.write("    Voxel size target: %sm\n" % voxel_size_target)
# LOA_file.write("    Raw filtering C2C-distance: %sm\n" %c2c_treshold)
# LOA_file.write("    Initial search radius for neighbours: %sm\n" % init_radius)
# LOA_file.write("    Maximum search radius: %sm\n\n" % search_treshold)
# LOA_file.write("    Total number of points: %s\n" %(len(Final_inlier_pcd.points) + len(Clutter_pcd.points)))
# prct_clutter = round(len(Clutter_pcd.points)/(len(Final_inlier_pcd.points) + len(Clutter_pcd.points))*100)
# LOA_file.write("        Total number of points calssified as clutter: %s  =>  %s%%\n\n\n" %(len(Clutter_pcd.points),prct_clutter))

# LOA_file.write("II. overall LOA specifications (Clutter incl.):\n\n")
# Total_points = len(Final_inlier_pcd.points) + len(Clutter_pcd.points)
# LOA30 = Class_0_LOA30 + Class_1_LOA30 + Class_2_LOA30 + Class_3_LOA30 + Class_4_LOA30
# prcte_LOA30 = LOA30/Total_points*100
# LOA_file.write("    LOA30: %s / %s  => %s %% \n" %(LOA30, Total_points, prcte_LOA30))
# LOA20 = Class_0_LOA20 + Class_1_LOA20 + Class_2_LOA20 + Class_3_LOA20 + Class_4_LOA20
# prcte_LOA20 = LOA20/Total_points*100
# LOA_file.write("    LOA20: %s / %s  => %s %% \n" %(LOA20, Total_points, prcte_LOA20))
# LOA10 = Class_0_LOA10 + Class_1_LOA10 + Class_2_LOA10 + Class_3_LOA10 + Class_4_LOA10
# prcte_LOA10 = LOA10/Total_points*100
# LOA_file.write("    LOA10: %s / %s  => %s %% \n\n\n" %(LOA10, Total_points, prcte_LOA10))

# LOA_file.write("III. overall LOA specifications (Clutter excl.):\n\n")
# Total_points = len(Final_inlier_pcd.points)
# LOA30 = Class_0_LOA30 + Class_1_LOA30 + Class_2_LOA30 + Class_3_LOA30 + Class_4_LOA30
# prcte_LOA30 = LOA30/Total_points*100
# LOA_file.write("    LOA30: %s / %s  => %s %% \n" %(LOA30, Total_points, prcte_LOA30))
# LOA20 = Class_0_LOA20 + Class_1_LOA20 + Class_2_LOA20 + Class_3_LOA20 + Class_4_LOA20
# prcte_LOA20 = LOA20/Total_points*100
# LOA_file.write("    LOA20: %s / %s  => %s %% \n" %(LOA20, Total_points, prcte_LOA20))
# LOA10 = Class_0_LOA10 + Class_1_LOA10 + Class_2_LOA10 + Class_3_LOA10 + Class_4_LOA10
# prcte_LOA10 = LOA10/Total_points*100
# LOA_file.write("    LOA10: %s / %s  => %s %% \n\n\n" %(LOA10, Total_points, prcte_LOA10))


# LOA_file.write("VI. By class LOA specifications (Clutter excl.):\n\n")

# LOA_file.write("    Class 0:  %s \n" % label_id[0])
# Total_points = Class_0_points
# LOA30 = Class_0_LOA30
# prcte_LOA30 = LOA30/Total_points*100
# LOA_file.write("        LOA30: %s / %s  => %s %% \n" %(LOA30, Total_points, prcte_LOA30))
# LOA20 = Class_0_LOA20
# prcte_LOA20 = LOA20/Total_points*100
# LOA_file.write("        LOA20: %s / %s  => %s %% \n" %(LOA20, Total_points, prcte_LOA20))
# LOA10 = Class_0_LOA10
# prcte_LOA10 = LOA10/Total_points*100
# LOA_file.write("        LOA10: %s / %s  => %s %% \n\n" %(LOA10, Total_points, prcte_LOA10))

# LOA_file.write("    Class 1:  %s \n" % label_id[1])
# Total_points = Class_1_points
# LOA30 = Class_1_LOA30
# prcte_LOA30 = LOA30/Total_points*100
# LOA_file.write("        LOA30: %s / %s  => %s %% \n" %(LOA30, Total_points, prcte_LOA30))
# LOA20 = Class_1_LOA20
# prcte_LOA20 = LOA20/Total_points*100
# LOA_file.write("        LOA20: %s / %s  => %s %% \n" %(LOA20, Total_points, prcte_LOA20))
# LOA10 = Class_1_LOA10
# prcte_LOA10 = LOA10/Total_points*100
# LOA_file.write("        LOA10: %s / %s  => %s %% \n\n" %(LOA10, Total_points, prcte_LOA10))

# LOA_file.write("    Class 2:  %s \n" % label_id[2])
# Total_points = Class_2_points
# LOA30 = Class_2_LOA30
# prcte_LOA30 = LOA30/Total_points*100
# LOA_file.write("        LOA30: %s / %s  => %s %% \n" %(LOA30, Total_points, prcte_LOA30))
# LOA20 = Class_2_LOA20
# prcte_LOA20 = LOA20/Total_points*100
# LOA_file.write("        LOA20: %s / %s  => %s %% \n" %(LOA20, Total_points, prcte_LOA20))
# LOA10 = Class_2_LOA10
# prcte_LOA10 = LOA10/Total_points*100
# LOA_file.write("        LOA10: %s / %s  => %s %% \n\n" %(LOA10, Total_points, prcte_LOA10))

# LOA_file.write("    Class 3:  %s \n" % label_id[3])
# Total_points = Class_3_points
# LOA30 = Class_3_LOA30
# prcte_LOA30 = LOA30/Total_points*100
# LOA_file.write("        LOA30: %s / %s  => %s %% \n" %(LOA30, Total_points, prcte_LOA30))
# LOA20 = Class_3_LOA20
# prcte_LOA20 = LOA20/Total_points*100
# LOA_file.write("        LOA20: %s / %s  => %s %% \n" %(LOA20, Total_points, prcte_LOA20))
# LOA10 = Class_3_LOA10
# prcte_LOA10 = LOA10/Total_points*100
# LOA_file.write("        LOA10: %s / %s  => %s %% \n\n" %(LOA10, Total_points, prcte_LOA10))

# LOA_file.write("    Class 4:  %s \n" % label_id[4])
# Total_points = Class_4_points
# LOA30 = Class_4_LOA30
# prcte_LOA30 = LOA30/Total_points*100
# LOA_file.write("        LOA30: %s / %s  => %s %% \n" %(LOA30, Total_points, prcte_LOA30))
# LOA20 = Class_4_LOA20
# prcte_LOA20 = LOA20/Total_points*100
# LOA_file.write("        LOA20: %s / %s  => %s %% \n" %(LOA20, Total_points, prcte_LOA20))
# LOA10 = Class_4_LOA10
# prcte_LOA10 = LOA10/Total_points*100
# LOA_file.write("        LOA10: %s / %s  => %s %% \n\n" %(LOA10, Total_points, prcte_LOA10))

# t10 = time.perf_counter()
# dt10 = t10 - t9
# print("LOA specs writen in %s s" % dt10)
# logging.info("LOA specs writen in %s s" % dt10)

In [41]:
# timing_file = open(timing_location, "w")

In [42]:

# timing_file = open(timing_location, "w")
# timing_file.write("PROCESS TIMING\n\n")

# total_time = dt1 + dt2 + dt3 + dt4 + dt5 + dt6 + dt7 + dt8 + dt9 + dt10

# print("Importing + sampling meshes for reference: %s s" % dt1)
# timing_file.write("Importing + sampling meshes for reference:\n")
# hours = math.floor(dt1/3600)
# minutes = math.floor(dt1/60 - hours*60)
# seconds = dt1 - hours*3600 - minutes*60
# timing_file.write("     %sh %smin %ss\n     %s %%\n" %(hours, minutes, seconds, round(dt1/total_time*100)))

# print("Created reference cloud from meshes: %s s" % dt2)
# timing_file.write("Created reference cloud from meshes:\n")
# hours = math.floor(dt2/3600)
# minutes = math.floor(dt2/60 - hours*60)
# seconds = dt2 - hours*3600 - minutes*60
# timing_file.write("     %sh %smin %ss\n     %s %%\n" %(hours, minutes, seconds, round(dt2/total_time*100)))

# print("Loading Target cloud: %s s" % dt3)
# timing_file.write("Loading Target cloud:\n")
# hours = math.floor(dt3/3600)
# minutes = math.floor(dt3/60 - hours*60)
# seconds = dt3 - hours*3600 - minutes*60
# timing_file.write("     %sh %smin %ss\n     %s %%\n" %(hours, minutes, seconds, round(dt3/total_time*100)))

# print("Downsampling Target pointcloud: %s s" % dt4)
# timing_file.write("Downsampling Target pointcloud:\n")
# hours = math.floor(dt4/3600)
# minutes = math.floor(dt4/60 - hours*60)
# seconds = dt4 - hours*3600 - minutes*60
# timing_file.write("     %sh %smin %ss\n     %s %%\n" %(hours, minutes, seconds, round(dt4/total_time*100)))

# print("Cloud 2 cloud calculation: %s s" % dt5)
# timing_file.write("Cloud 2 cloud calculation:\n" )
# hours = math.floor(dt5/3600)
# minutes = math.floor(dt5/60 - hours*60)
# seconds = dt5 - hours*3600 - minutes*60
# timing_file.write("     %sh %smin %ss\n     %s %%\n" %(hours, minutes, seconds, round(dt5/total_time*100)))

# print("Raw filtering: %s s" % dt6)
# timing_file.write("Raw filtering:\n")
# hours = math.floor(dt6/3600)
# minutes = math.floor(dt6/60 - hours*60)
# seconds = dt6 - hours*3600 - minutes*60
# timing_file.write("     %sh %smin %ss\n     %s %%\n" %(hours, minutes, seconds, round(dt6/total_time*100)))

# print("Fine filtering: %s s" % dt7)
# timing_file.write("Fine filtering:\n")
# hours = math.floor(dt7/3600)
# minutes = math.floor(dt7/60 - hours*60)
# seconds = dt7 - hours*3600 - minutes*60
# timing_file.write("     %sh %smin %ss\n     %s %%\n" %(hours, minutes, seconds, round(dt7/total_time*100)))

# print("Calculating LOA specs: %s s" % dt8)
# timing_file.write("Calculating LOA specs:\n")
# hours = math.floor(dt8/3600)
# minutes = math.floor(dt8/60 - hours*60)
# seconds = dt8 - hours*3600 - minutes*60
# timing_file.write("     %sh %smin %ss\n     %s %%\n" %(hours, minutes, seconds, round(dt8/total_time*100)))

# print("Saving clouds: %s s" % dt9)
# timing_file.write("Saving clouds:\n")
# hours = math.floor(dt9/3600)
# minutes = math.floor(dt9/60 - hours*60)
# seconds = dt9 - hours*3600 - minutes*60
# timing_file.write("     %sh %smin %ss\n     %s %%\n" %(hours, minutes, seconds, round(dt9/total_time*100)))

# print("writing LOA specs: %s s" % dt10)
# timing_file.write("writing LOA specs:\n")
# hours = math.floor(dt10/3600)
# minutes = math.floor(dt10/60 - hours*60)
# seconds = dt10 - hours*3600 - minutes*60
# timing_file.write("     %sh %smin %ss\n     %s %%\n" %(hours, minutes, seconds, round(dt10/total_time*100)))

# print("TOTAL: %s s" % total_time)
# timing_file.write("TOTAL: \n")
# hours = math.floor(total_time/3600)
# minutes = math.floor(total_time/60 - hours*60)
# seconds = total_time - hours*3600 - minutes*60
# timing_file.write("     %sh %smin %ss\n" %(hours, minutes, seconds))

In [43]:
o3d.visualization.draw_geometries([Final_inlier_pcd,Clutter_pcd])